In [1]:
# !pip install transformers
from transformers import pipeline
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


GPU is available


In [3]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/EBAC Sem4/practice project/100k.csv')
# df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/EBAC Sem4/practice project/PLP.csv')
df

Mounted at /content/drive


,author,body,created_utc,id,submission,subreddit,subreddit_id
0,ForeskinStealer420,A combination of (1) and (2) would make you th...,2024-09-14 22:40:45,ln5r3zv,1fgx26q,datascience,t5_2sptq
1,lakeland_nz,4. Shift to something less technically. Becom...,2024-09-14 22:54:52,ln5tqg8,1fgx26q,datascience,t5_2sptq
2,Moscow_Gordon,Maybe look into solution architect positions f...,2024-09-15 00:16:38,ln67tl2,1fgx26q,datascience,t5_2sptq
3,Numerous-Tip-5097,"Hey, I am on the same boat with less experienc...",2024-09-15 01:37:26,ln6jubq,1fgx26q,datascience,t5_2sptq
4,hornswoggle_choco05,I'd probably shake things up and try a new pat...,2024-09-15 01:58:02,ln6mtyc,1fgx26q,datascience,t5_2sptq
...,...,...,...,...,...,...,...
107716,RatsOnCocaine69,"Hey, I'm a cyber rookie! Could you please expl...",2024-09-02 16:35:00,ll62p1s,1f73hmj,cybersecurity,t5_2u559
107717,identicalBadger,If let’s encrypt can reliably secure consumer ...,2024-09-02 16:53:15,ll661lt,1f73hmj,cybersecurity,t5_2u559
107718,ExcitedForNothing,Not the person you were replying to but here g...,2024-09-02 17:10:30,ll697hq,1f73hmj,cybersecurity,t5_2u559
107719,whatThisOldThrowAway,I was just giving examples of activities that ...,2024-09-02 17:53:40,ll6h292,1f73hmj,cybersecurity,t5_2u559


In [4]:
############ using sentiment analysis model

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device = 0)
# # List to store the results
# results = []
# def truncate_to_max_length(text, max_length):
#     encoded_input = tokenizer(text, truncation=True, max_length=max_length, return_tensors='pt')
#     return tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)

# for index, row in tqdm(df.iterrows(), total=len(df), ncols=100):
#     review = row['body']
#     subreddit = row['subreddit']
#     created_utc = row['created_utc']


#     # Sentence tokenization
#     for sentence in sent_tokenize(review):
#         truncated_sentence = truncate_to_max_length(sentence, max_length=512)
#         # Get sentiment for the sentence
#         sentiment_result = sentiment_analyzer(truncated_sentence)[0]

#         # Extract sentiment label and score
#         sentiment_label = sentiment_result['label']
#         sentiment_score = sentiment_result['score']

#         # Append results
#         results.append({
#             "subreddit": subreddit,
#             "created_utc": created_utc,
#             "sentence": sentence,
#             "sentiment": sentiment_label,
#             "sentiment_score": sentiment_score
#         })


In [5]:
################ using NLI zero shot classifier

tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
sentiment_analyzer = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=0)
candidate_labels = ["POSITIVE", "NEUTRAL", "NEGATIVE"]

# List to store the results
results = []

def truncate_to_max_length(text, max_length):
    encoded_input = tokenizer(text, truncation=True, max_length=max_length, return_tensors='pt')
    return tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)

for index, row in tqdm(df.iterrows(), total=len(df), ncols=100):
    review = row['body']
    subreddit = row['subreddit']
    created_utc = row['created_utc']

    # Sentence tokenization
    for sentence in sent_tokenize(review):
        truncated_sentence = truncate_to_max_length(sentence, max_length=512)

        # Get sentiment for the sentence using zero-shot classification
        sentiment_result = sentiment_analyzer(truncated_sentence, candidate_labels)

        # Append results
        results.append({
            "subreddit": subreddit,
            "created_utc": created_utc,
            "sentence": sentence,
            "labels": sentiment_result['labels'],
            "scores": sentiment_result['scores']
        })

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|█████████████████████████████████████████████████████| 107721/107721 [5:04:30<00:00,  5.90it/s]


In [6]:
sentiment_df = pd.DataFrame(results)
sentiment_df.to_csv('/content/drive/My Drive/Colab Notebooks/EBAC Sem4/practice project/sentiment_analysis_results_roberta.csv', index=False)
print("Sentiment analysis results saved")

Sentiment analysis results saved
